# Melon 주간 차트 크롤링

In [1]:
import requests
import pandas as pd

In [6]:
# 지니차트
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [8]:
# 한빛 아카데미
url = 'https://www.hanbit.co.kr/academy/'
req = requests.get(url)
html = req.text
# html

In [5]:
# 멜론차트
url = 'https://www.melon.com/chart/week/index.htm'
req = requests.get(url)
html = req.text
html # '' ; 일부사이트는 데이터 불러오지 못함.

''

In [10]:
# 데이터 불러오지 못하는 일부 사이트에 대하여 작업
# header에 'User-Agent'값을 주어야만 데이터를 가져올수 있음
# 멜론차트
url = 'https://www.melon.com/chart/week/index.htm'
header = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
req = requests.get(url, headers = header)
html = req.text
#html 

In [12]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser') #멜론차트를 request 이용해 html로 가져온 상황
#soup

 ### Selector
 
    lst50 > td:nth-child(6) > div > div > div.ellipsis.rank02 > a

 ### Xpath 

    //*[@id="lst50"]/td[6]/div/div/div[2]/a 

In [17]:
trs = soup.select("#lst50") # id명으로 가져옴:#
len(trs)

50

In [21]:
tr = trs[0]
tr

<tr class="lst50" data-song-no="33507137" id="lst50">
<td><div class="wrap t_right"><input class="input_check" name="input_check" title="Butter 곡 선택" type="checkbox" value="33507137"/></div></td>
<td><div class="wrap t_center"><span class="rank">1</span><span class="none">위</span></div></td>
<!-- 차트순위 추가 -->
<td><div class="wrap">
<span class="rank_wrap" title="14단계 상승">
<span class="bullet_icons rank_up"><span class="none">단계 상승</span></span>
<span class="up">14</span>
</span>
</div></td>
<td><div class="wrap">
<a class="image_typeAll" href="javascript:melon.link.goAlbumDetail('10612483');" title="Butter">
<img alt="Butter - 페이지 이동" height="60" onerror="WEBPOCIMG.defaultAlbumImg(this);" src="https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize" width="60">
<span class="bg_album_frame"></span>
</img></a>
</div></td>
<td><div class="wrap">
<a class="btn button_icons type03 song_info" href="javascript:melon.link.goSon

In [28]:
# rank데이터 가져오기
rank = tr.select_one('.rank').string #; rank  # 문자형이므로
rank = int(rank) ; rank

1

In [26]:
rank = tr.select_one('.rank').get_text; rank

<bound method Tag.get_text of <span class="rank">1</span>>

In [33]:
# 곡명 찾아오기
title = tr.select_one(".ellipsis.rank01"); title
# class 명에 space bar 가 존재할시 ""안이더라도 .으로 바꾸어 주어야한다!!

<div class="ellipsis rank01"><span>
<a href="javascript:melon.play.playSong('19041301',33507137);" title="Butter 재생">Butter</a>
</span></div>

In [36]:
title = tr.select_one(".ellipsis.rank01")
title = title.find('a').get_text() # <a> tag의 text를 가져올것
title

'Butter'

In [41]:
# 가수명 찾아오기
artist = tr.select_one(".ellipsis.rank02"); artist

<div class="ellipsis rank02">
<a href="javascript:melon.link.goArtistDetail('672375');" title="방탄소년단 - 페이지 이동">방탄소년단</a><span class="checkEllipsis" style="display:none"><a href="javascript:melon.link.goArtistDetail('672375');" title="방탄소년단 - 페이지 이동">방탄소년단</a></span>
</div>

In [42]:
artist = artist.find('a').get_text()
artist

'방탄소년단'

In [43]:
# 앨범
album  = tr.select_one(".ellipsis.rank03"); album
album = album.find('a').string # get_text()대신 string 도 가능
album

'Butter'

### 50곡에 대해서 데이터 프레임 만들기

In [45]:
rank_list, title_list, artist_list, album_list = [],[],[],[]
for tr in trs:
    rank = tr.select_one('.rank').string 
    rank = int(rank) 
    title = tr.select_one(".ellipsis.rank01")
    title = title.find('a').get_text() 
    artist = tr.select_one(".ellipsis.rank02")
    artist = artist.find('a').get_text()
    album  = tr.select_one(".ellipsis.rank03")
    album = album.find('a').string

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

In [47]:
df = pd.DataFrame({
    '순위' : rank_list,
    '곡명' : title_list,
    '가수': artist_list,
    '앨범':album_list
})
df.head()

,순위,곡명,가수,앨범
0,1,Butter,방탄소년단,Butter
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL
2,3,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice
3,4,헤픈 우연,헤이즈 (Heize),HAPPEN
4,5,롤린 (Rollin'),브레이브걸스,Rollin'


### 이미지 속성값 가져오기

In [51]:
image = trs[0].select_one('.image_typeAll').find('img') # img태그 가져옴
image = image['src'] # 속성값 가져오기
image

'https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize'

### - QUIZ]

In [55]:
trs50 = soup.select("#lst50") 
trs100 = soup.select("#lst100") 
len(trs50) , len(trs100)

(50, 50)

In [61]:
rank_list, title_list, artist_list, album_list, image_list = [],[],[],[],[]
# top 50
for tr in trs50:
    rank = tr.select_one('.rank').string 
    rank = int(rank) 
    title = tr.select_one(".ellipsis.rank01")
    title = title.find('a').get_text() 
    artist = tr.select_one(".ellipsis.rank02")
    artist = artist.find('a').get_text()
    album  = tr.select_one(".ellipsis.rank03")
    album = album.find('a').string
    image = trs[0].select_one('.image_typeAll').find('img') 
    image = image['src'] 

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)
    image_list.append(image)

In [63]:
# 50위~100위
for tr in trs100:
    rank = tr.select_one('.rank').string 
    rank = int(rank) 
    title = tr.select_one(".ellipsis.rank01")
    title = title.find('a').get_text() 
    artist = tr.select_one(".ellipsis.rank02")
    artist = artist.find('a').get_text()
    album  = tr.select_one(".ellipsis.rank03")
    album = album.find('a').string
    image = trs[0].select_one('.image_typeAll').find('img') 
    image = image['src'] 

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)
    image_list.append(image)

In [64]:
df = pd.DataFrame({
    '순위' : rank_list,
    '곡명' : title_list,
    '가수': artist_list,
    '앨범': album_list,
    '사진': image_list
})
df.head()

,순위,곡명,가수,앨범,사진
0,1,Butter,방탄소년단,Butter,https://cdnimg.melon.co.kr/cm2/album/images/10...
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL,https://cdnimg.melon.co.kr/cm2/album/images/10...
2,3,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice,https://cdnimg.melon.co.kr/cm2/album/images/10...
3,4,헤픈 우연,헤이즈 (Heize),HAPPEN,https://cdnimg.melon.co.kr/cm2/album/images/10...
4,5,롤린 (Rollin'),브레이브걸스,Rollin',https://cdnimg.melon.co.kr/cm2/album/images/10...


In [65]:
df.tail()

,순위,곡명,가수,앨범,사진
95,96,혼술하고 싶은 밤,벤,혼술하고 싶은 밤,https://cdnimg.melon.co.kr/cm2/album/images/10...
96,97,죄와벌,SG 워너비,살다가,https://cdnimg.melon.co.kr/cm2/album/images/10...
97,98,되풀이,임창정,펜트하우스2 OST Part.1,https://cdnimg.melon.co.kr/cm2/album/images/10...
98,99,고래 (Dive Into You),NCT DREAM,맛 (Hot Sauce) - The 1st Album,https://cdnimg.melon.co.kr/cm2/album/images/10...
99,100,돌림노래 (Feat. DEAN),아이유,IU 5th Album 'LILAC',https://cdnimg.melon.co.kr/cm2/album/images/10...


### sol]

In [68]:
type(trs)

bs4.element.ResultSet

In [67]:
trs = soup.select("#lst50") 
trs2 = soup.select("#lst100") 
trs.extend(trs2)  # bs4.element.ResultSet 합치기
len(trs)

100

In [69]:
rank_list, title_list, artist_list, album_list, image_list = [],[],[],[],[]
# top 100
for tr in trs:
    rank = tr.select_one('.rank').string 
    rank = int(rank) 
    title = tr.select_one(".ellipsis.rank01")
    title = title.find('a').get_text() 
    artist = tr.select_one(".ellipsis.rank02")
    artist = artist.find('a').get_text()
    album  = tr.select_one(".ellipsis.rank03")
    album = album.find('a').string
    image = trs[0].select_one('.image_typeAll').find('img') 
    image = image['src'] 

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)
    image_list.append(image)

In [71]:
df = pd.DataFrame({
    '순위' : rank_list,
    '곡명' : title_list,
    '가수': artist_list,
    '앨범': album_list,
    '사진': image_list
})
df.head()

,순위,곡명,가수,앨범,사진
0,1,Butter,방탄소년단,Butter,https://cdnimg.melon.co.kr/cm2/album/images/10...
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL,https://cdnimg.melon.co.kr/cm2/album/images/10...
2,3,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice,https://cdnimg.melon.co.kr/cm2/album/images/10...
3,4,헤픈 우연,헤이즈 (Heize),HAPPEN,https://cdnimg.melon.co.kr/cm2/album/images/10...
4,5,롤린 (Rollin'),브레이브걸스,Rollin',https://cdnimg.melon.co.kr/cm2/album/images/10...


In [72]:
df.tail()

,순위,곡명,가수,앨범,사진
95,96,혼술하고 싶은 밤,벤,혼술하고 싶은 밤,https://cdnimg.melon.co.kr/cm2/album/images/10...
96,97,죄와벌,SG 워너비,살다가,https://cdnimg.melon.co.kr/cm2/album/images/10...
97,98,되풀이,임창정,펜트하우스2 OST Part.1,https://cdnimg.melon.co.kr/cm2/album/images/10...
98,99,고래 (Dive Into You),NCT DREAM,맛 (Hot Sauce) - The 1st Album,https://cdnimg.melon.co.kr/cm2/album/images/10...
99,100,돌림노래 (Feat. DEAN),아이유,IU 5th Album 'LILAC',https://cdnimg.melon.co.kr/cm2/album/images/10...


### 파일에 저장하기

In [79]:
period = soup.select_one('.yyyymmdd').get_text().strip()
period

'2021.05.24 ~ 2021.05.30'

In [83]:
filename = f'melon({period}).csv'
df.to_csv(filename, index =False, sep = ',', encoding= 'utf8')